In [1]:
import os
import io
import pandas as pd
from google.cloud import storage
import ast
import matplotlib.pyplot as plt 
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

key2 = os.getenv('GCP_KEY_2')

client2 = storage.Client.from_service_account_json(key2)

##### 2023-04-30

In [2]:
log1 = pd.read_parquet('gs://sprint_final_project/final_project/hackle_log/2023-04-30.parquet', engine='pyarrow')

In [7]:
log1.head()

,Asia/Seoul,event_key,session_id,user_id,value,user_properties,hackle_properties,event_properties,id,device_id
0,2023-05-01T00:00:00.024Z,button,jgGFnoyc6GWuGIEgjp3nUoKRQbF3,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...","{'description': '다음', 'location': 'signotherin...",96d1336f-744b-4deb-a6a4-207494419dbf,cfc34eab-6930-488b-8109-70d37cec7dae
1,2023-05-01T00:00:00.025Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,None,0.0,[],"{'language': 'ko-CA', 'isapp': 'true', 'osvers...","{'description': 'vote 선택', 'location': 'homevo...",2825d1b1-8325-42e2-88eb-8cae61d3a26d,0dde2ecb-ff03-440f-98f3-94f20978b796
2,2023-05-01T00:00:00.029Z,button,L31Dl8vRtdWPJ8XC0gGs1mPwzJH2,None,0.0,[],"{'language': 'ko', 'isapp': 'true', 'osversion...","{'description': '친구선택', 'location': 'homeenque...",335b0a36-9ecf-41a9-b7d2-e27125e521ef,b50774ed-24ce-49e1-9383-945ff3aea9ff
3,2023-05-01T00:00:00.036Z,button,QjToElcYNkVxszJDHBhqtITlzDJ3,None,0.0,[],"{'language': 'ko', 'isapp': 'true', 'osversion...","{'description': '이름 셔플!', 'location': 'homeenq...",b730bb6a-589a-4074-89a1-87f11a03f9d6,0999c6b6-c579-4c0f-916c-9a113010f84a
4,2023-05-01T00:00:00.063Z,button,VRpXf303RGU730fIO4EjWstzAJi1,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...","{'description': '앱바 뒤로가기', 'location': 'homefr...",c2a5fd3f-92d5-4864-8e0e-63e563898499,45985493-ed35-4cc5-935f-1bcf922e213c


In [3]:
log1_1 = log1[log1.index < 200000]

In [4]:
log1_1['user_properties'] = log1_1['user_properties'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

In [6]:
log1_1_1 = log1_1.explode('user_properties')
log1_1_1['user_properties'].nunique()

0

In [5]:
log1_1 = log1_1.drop(columns=['value','user_id','id','device_id','user_properties'])

In [6]:
def expand_dict_columns(df, dict_columns):
    df = df.copy()
    for col in dict_columns:
        # # 문자열인 dict 컬럼을 실제 dict로 변환
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

        # dict 컬럼을 여러 개의 컬럼으로 확장
        expanded = df[col].apply(pd.Series)

        # 컬럼 이름에 원래 컬럼명을 prefix로 추가
        expanded.columns = [f"{col}_{k}" for k in expanded.columns]

        # 원래 컬럼 제거하고 확장된 컬럼 병합
        df = pd.concat([df.drop(columns=[col]), expanded], axis=1)
    return df

log1_2 = expand_dict_columns(log1_1, ['hackle_properties', 'event_properties'])

In [19]:
log1_2 = log1_2.drop(columns=['hackle_properties_language','hackle_properties_osversion','hackle_properties_devicevendor','hackle_properties_osname','hackle_properties_versionname','hackle_properties_platform','hackle_properties_devicemodel','hackle_properties_isapp'])

KeyError: "['hackle_properties_language', 'hackle_properties_osversion', 'hackle_properties_devicevendor', 'hackle_properties_osname', 'hackle_properties_versionname', 'hackle_properties_platform', 'hackle_properties_devicemodel'] not found in axis"

In [22]:
log1_2

,Asia/Seoul,event_key,session_id,event_properties_description,event_properties_location,event_properties_sessionid
0,2023-05-01T00:00:00.024Z,button,jgGFnoyc6GWuGIEgjp3nUoKRQbF3,다음,signotherinputscreen,None
1,2023-05-01T00:00:00.025Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,vote 선택,homevotesscreen,None
2,2023-05-01T00:00:00.029Z,button,L31Dl8vRtdWPJ8XC0gGs1mPwzJH2,친구선택,homeenquetescreen,None
3,2023-05-01T00:00:00.036Z,button,QjToElcYNkVxszJDHBhqtITlzDJ3,이름 셔플!,homeenquetescreen,None
4,2023-05-01T00:00:00.063Z,button,VRpXf303RGU730fIO4EjWstzAJi1,앱바 뒤로가기,homefriendprofilescreen,None
...,...,...,...,...,...,...
199995,2023-05-01T00:53:05.393Z,button,MbS10unFHUQlYtNE4v4yZGAaxSz2,내 실명은 ‘김예진' 맞아!,signotherinputscreen,None
199996,2023-05-01T00:53:05.423Z,button,AWaY7FMKtrcUxNlSf8i9ZRE12mv1,vote start,homeidlescreen,None
199997,2023-05-01T00:53:05.449Z,$session_start,f2pzw9aP0HMw9UiJdiW2HBjYKjk1,None,None,1682869985449.6f4d7b9c
199998,2023-05-01T00:53:05.480Z,button,qUdI3IvIlVUCCJyNLUTCNDGSdIx1,앱바 뒤로가기,homedetailscreen,None


In [23]:
log1_2['event_key'].unique()

array(['button', '$session_end', '$session_start'], dtype=object)

In [24]:
log1_2['event_properties_description'].unique()

array(['다음', 'vote 선택', '친구선택', '이름 셔플!', '앱바 뒤로가기', 'votes', None,
       '타임라인', '확인', '건너뛰기', '프로필', '친구+', '여자', '초성 힌트 받기', 'vote start',
       '학교 선택', '앱바 닫기', '친구 toggle', '친구 초대하고 바로 받기', '홈', '닫기', '홈으로',
       '사진첩에서 추가', '전체 이름 확인하기', '프로필 사진 수정', '계속하기', '칭찬 알림 받기', '학교',
       '동의하고 계속', 'title', '사진첩에서 불러오기', '인스타그램', '자동으로 친구추가', '문의하기',
       'feedback', "내 실명은 ‘홍유성' 맞아!", "내 실명은 ‘김주현' 맞아!", '내 학교 찾아보기',
       '친구 끊기', '취소', '다시 선택', "내 실명은 ‘박선우' 맞아!", 'about',
       "내 실명은 ‘김태은' 맞아!", '이미 가입했다면 로그인!', '설정으로 이동하기', '계정삭제',
       '연락처 불러오기', '로그아웃', '초기화하고 첫 화면으로', '초대링크 복사하기!',
       "내 실명은 ‘이수연' 맞아!", '아이디', '남자', '성별', '프로필 사진 삭제하기', '다시 입력하기',
       "내 실명은 ‘김효구' 맞아!", "내 실명은 ‘박혜원' 맞아!", '정말 떠날거니?', '함께 할 친구가 없어서',
       'vote 시작', "내 실명은 ‘최성율' 맞아!", '이름', '재밌는 질문이 없어서', '첫 화면 으로',
       '계정 삭제하기', '초대 링크 복사', "내 실명은 ‘김유건' 맞아!", "내 실명은 ‘최민석' 맞아!",
       "내 실명은 ‘고명선' 맞아!", "내 실명은 ‘서윤진' 맞아!", '정말로 로그아웃 할거야?',
       "내 실명은 ‘김서연' 맞아!", "내 실명은 ‘장승영' 맞아!", "내 

In [27]:
log1_2[log1_2['session_id']=='ndyQc1KhirW2dMSGRscaXhV0NLx1']

,Asia/Seoul,event_key,session_id,event_properties_description,event_properties_location,event_properties_sessionid
9,2023-05-01T00:00:00.149Z,button,ndyQc1KhirW2dMSGRscaXhV0NLx1,타임라인,homescreen,None
193,2023-05-01T00:00:02.978Z,button,ndyQc1KhirW2dMSGRscaXhV0NLx1,votes,homescreen,None
295,2023-05-01T00:00:04.612Z,button,ndyQc1KhirW2dMSGRscaXhV0NLx1,vote 선택,homevotesscreen,None
1133,2023-05-01T00:00:16.331Z,button,ndyQc1KhirW2dMSGRscaXhV0NLx1,앱바 뒤로가기,homedetailscreen,None
18864,2023-05-01T00:04:13.105Z,button,ndyQc1KhirW2dMSGRscaXhV0NLx1,앱바 뒤로가기,homefriendprofilescreen,None
19881,2023-05-01T00:04:26.940Z,button,ndyQc1KhirW2dMSGRscaXhV0NLx1,앱바 뒤로가기,homefriendprofilescreen,None
20591,2023-05-01T00:04:36.373Z,button,ndyQc1KhirW2dMSGRscaXhV0NLx1,앱바 뒤로가기,homefriendprofilescreen,None
22576,2023-05-01T00:05:02.139Z,button,ndyQc1KhirW2dMSGRscaXhV0NLx1,앱바 뒤로가기,homefriendprofilescreen,None
23111,2023-05-01T00:05:09.304Z,button,ndyQc1KhirW2dMSGRscaXhV0NLx1,앱바 뒤로가기,homefriendprofilescreen,None
23719,2023-05-01T00:05:17.272Z,button,ndyQc1KhirW2dMSGRscaXhV0NLx1,앱바 뒤로가기,homefriendprofilescreen,None


In [31]:
log1_2[log1_2['session_id']=='n3PI8GZnRFOKpJslIRgckMeRiln1'].head(60)

,Asia/Seoul,event_key,session_id,event_properties_description,event_properties_location,event_properties_sessionid
1,2023-05-01T00:00:00.025Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,vote 선택,homevotesscreen,None
79,2023-05-01T00:00:01.261Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,앱바 뒤로가기,homedetailscreen,None
135,2023-05-01T00:00:02.109Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,vote 선택,homevotesscreen,None
258,2023-05-01T00:00:03.926Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,앱바 뒤로가기,homedetailscreen,None
293,2023-05-01T00:00:04.589Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,vote 선택,homevotesscreen,None
454,2023-05-01T00:00:06.856Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,앱바 뒤로가기,homedetailscreen,None
534,2023-05-01T00:00:08.036Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,타임라인,homescreen,None
682,2023-05-01T00:00:10.085Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,친구+,homescreen,None
165618,2023-05-01T00:42:15.309Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,vote start,homeidlescreen,None
165839,2023-05-01T00:42:18.960Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,이름 셔플!,homeenquetescreen,None


##### 2023-05-06

In [ ]:
log2 = pd.read_parquet('gs://sprint_final_project/final_project/hackle_log/2023-05-06.parquet', engine='pyarrow')

##### 2023-05-12

In [ ]:
log3 = pd.read_parquet('gs://sprint_final_project/final_project/hackle_log/2023-05-12.parquet', engine='pyarrow')

##### 2023-05-18

In [ ]:
log4 = pd.read_parquet('gs://sprint_final_project/final_project/hackle_log/2023-05-18.parquet', engine='pyarrow')

##### 2023-05-24

In [2]:
log5 = pd.read_parquet('gs://sprint_final_project/final_project/hackle_log/2023-05-24.parquet', engine='pyarrow')

In [11]:
log5['event_key'].nunique()

1

In [3]:
log5_1 = log5[log5.index < 200000]

In [4]:
log5_1['user_properties'] = log5_1['user_properties'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

In [5]:
log5_1 = log5_1.drop(columns=['value','id','device_id','user_properties'])
log5_1

,Asia/Seoul,event_key,session_id,user_id,hackle_properties,event_properties
0,2023-05-25T00:00:00.219Z,$session_start,TeeVVRGrpuXE5q6f7pAu9ULVNvG3,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684940400219.62ee4573'}
1,2023-05-25T00:00:00.224Z,$session_start,9tcpOeHvyldCUXMgRyFVQI86rf13,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684940400224.df939c47'}
2,2023-05-25T00:00:00.275Z,$session_start,aiDrtN8uHiQXRXmLMJhf27hKA4B3,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684940400275.457b984b'}
3,2023-05-25T00:00:00.369Z,$session_start,Oqmt3cCrCQZAa0TT3B4s2kTOG1k1,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684940400369.db2cd3ea'}
4,2023-05-25T00:00:00.493Z,$session_start,GA1Nz2ehK2TmsRcqBzXQn6hkDHH2,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684940400493.8e886f45'}
...,...,...,...,...,...,...
199995,2023-05-25T09:10:32.682Z,$session_start,a7rWG6CCv9QsM7RJYq4rkFz77Ss1,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684973432682.228da6a5'}
199996,2023-05-25T09:10:32.869Z,$session_start,zttLinV5jnhu6OzoyipzImtsjjB2,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684973432869.c19d3b39'}
199997,2023-05-25T09:10:33.037Z,$session_start,gu9TeaF0hWhTDga7QUQjqzaUYy82,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684973433037.a04ab327'}
199998,2023-05-25T09:10:33.048Z,$session_start,fUIFC8wJ6YaeK2kNhdEQs2ON1r22,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1684973433048.465aa95d'}


In [6]:
def expand_dict_columns(df, dict_columns):
    df = df.copy()
    for col in dict_columns:
        # # 문자열인 dict 컬럼을 실제 dict로 변환
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

        # dict 컬럼을 여러 개의 컬럼으로 확장
        expanded = df[col].apply(pd.Series)

        # 컬럼 이름에 원래 컬럼명을 prefix로 추가
        expanded.columns = [f"{col}_{k}" for k in expanded.columns]

        # 원래 컬럼 제거하고 확장된 컬럼 병합
        df = pd.concat([df.drop(columns=[col]), expanded], axis=1)
    return df

log5_2 = expand_dict_columns(log5_1, ['hackle_properties', 'event_properties'])

In [7]:
log5_2

,Asia/Seoul,event_key,session_id,user_id,hackle_properties_language,hackle_properties_isapp,hackle_properties_osversion,hackle_properties_devicevendor,hackle_properties_versionname,hackle_properties_osname,hackle_properties_platform,hackle_properties_devicemodel,event_properties_sessionid
0,2023-05-25T00:00:00.219Z,$session_start,TeeVVRGrpuXE5q6f7pAu9ULVNvG3,None,ko-KR,true,16.3,Apple,1.2.10,iOS,Mobile,"iPhone14,7",1684940400219.62ee4573
1,2023-05-25T00:00:00.224Z,$session_start,9tcpOeHvyldCUXMgRyFVQI86rf13,None,ko-KR,true,16.4.1,Apple,1.2.10,iOS,Mobile,"iPhone12,1",1684940400224.df939c47
2,2023-05-25T00:00:00.275Z,$session_start,aiDrtN8uHiQXRXmLMJhf27hKA4B3,None,ko-KR,true,16.4.1,Apple,1.2.10,iOS,Mobile,"iPhone12,1",1684940400275.457b984b
3,2023-05-25T00:00:00.369Z,$session_start,Oqmt3cCrCQZAa0TT3B4s2kTOG1k1,None,ko-KR,true,16.3.1,Apple,1.2.10,iOS,Mobile,"iPhone11,8",1684940400369.db2cd3ea
4,2023-05-25T00:00:00.493Z,$session_start,GA1Nz2ehK2TmsRcqBzXQn6hkDHH2,None,ko-KR,true,16.4.1,Apple,1.2.10,iOS,Mobile,"iPhone14,7",1684940400493.8e886f45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,2023-05-25T09:10:32.682Z,$session_start,a7rWG6CCv9QsM7RJYq4rkFz77Ss1,None,ko-KR,true,15.7.5,Apple,1.2.10,iOS,Mobile,"iPhone9,3",1684973432682.228da6a5
199996,2023-05-25T09:10:32.869Z,$session_start,zttLinV5jnhu6OzoyipzImtsjjB2,None,ko-KR,true,16.4.1,Apple,1.2.10,iOS,Mobile,"iPhone12,1",1684973432869.c19d3b39
199997,2023-05-25T09:10:33.037Z,$session_start,gu9TeaF0hWhTDga7QUQjqzaUYy82,None,ko-KR,true,16.3.1,Apple,1.2.10,iOS,Mobile,"iPhone14,5",1684973433037.a04ab327
199998,2023-05-25T09:10:33.048Z,$session_start,fUIFC8wJ6YaeK2kNhdEQs2ON1r22,None,ko-KR,true,16.4.1,Apple,1.2.10,iOS,Mobile,"iPhone14,5",1684973433048.465aa95d


In [8]:
log5_2 = log5_2.drop(columns=['hackle_properties_language','hackle_properties_osversion','hackle_properties_devicevendor',\
    'hackle_properties_osname','hackle_properties_versionname','hackle_properties_platform','hackle_properties_devicemodel',\
        'hackle_properties_isapp'])

In [13]:
log5_2

,Asia/Seoul,event_key,session_id,user_id,event_properties_sessionid
0,2023-05-25T00:00:00.219Z,$session_start,TeeVVRGrpuXE5q6f7pAu9ULVNvG3,None,1684940400219.62ee4573
1,2023-05-25T00:00:00.224Z,$session_start,9tcpOeHvyldCUXMgRyFVQI86rf13,None,1684940400224.df939c47
2,2023-05-25T00:00:00.275Z,$session_start,aiDrtN8uHiQXRXmLMJhf27hKA4B3,None,1684940400275.457b984b
3,2023-05-25T00:00:00.369Z,$session_start,Oqmt3cCrCQZAa0TT3B4s2kTOG1k1,None,1684940400369.db2cd3ea
4,2023-05-25T00:00:00.493Z,$session_start,GA1Nz2ehK2TmsRcqBzXQn6hkDHH2,None,1684940400493.8e886f45
...,...,...,...,...,...
199995,2023-05-25T09:10:32.682Z,$session_start,a7rWG6CCv9QsM7RJYq4rkFz77Ss1,None,1684973432682.228da6a5
199996,2023-05-25T09:10:32.869Z,$session_start,zttLinV5jnhu6OzoyipzImtsjjB2,None,1684973432869.c19d3b39
199997,2023-05-25T09:10:33.037Z,$session_start,gu9TeaF0hWhTDga7QUQjqzaUYy82,None,1684973433037.a04ab327
199998,2023-05-25T09:10:33.048Z,$session_start,fUIFC8wJ6YaeK2kNhdEQs2ON1r22,None,1684973433048.465aa95d


##### 2023-06-17

In [2]:
log6 = pd.read_parquet('gs://sprint_final_project/final_project/hackle_log/2023-06-17.parquet', engine='pyarrow')

In [3]:
log6_1 = log6[log6.index < 200000]

In [4]:
log6_1['user_properties'] = log6_1['user_properties'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

In [5]:
log6_1 = log6_1.drop(columns=['value','id','device_id','user_properties'])
log6_1

,Asia/Seoul,event_key,session_id,user_id,hackle_properties,event_properties
0,2023-06-18T00:07:08.014Z,$session_start,5cmi6ko54fPkNcJz6OftyN04nxL2,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
1,2023-06-18T00:07:11.039Z,$session_end,5cmi6ko54fPkNcJz6OftyN04nxL2,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
2,2023-06-18T00:07:34.065Z,$session_start,WrOTpJzMF0fUed9MlXQggRmyXbw2,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
3,2023-06-18T01:01:12.503Z,$session_start,nn7Kppgo3HfNTFkqiDCJsP2aOVI2,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
4,2023-06-18T02:30:47.469Z,$session_start,WrOTpJzMF0fUed9MlXQggRmyXbw2,None,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
...,...,...,...,...,...,...
199995,2023-06-21T09:57:11.762Z,view_message_tap,f7042063-bd63-4eaa-8347-47095ab58759,JDELTFGyC3annYUuLC1mIumKlJv1,"{'language': 'ko', 'isapp': 'true', 'osversion...",[]
199996,2023-06-21T09:57:11.887Z,view_home_tap,e47237e5-4d80-447a-a36b-a1864583990a,Ex4Ne8WkciTcwuobSOIMJeN9D1C2,"{'language': 'ko', 'isapp': 'true', 'osversion...",[]
199997,2023-06-21T09:57:12.261Z,$session_start,vBiRfuO1c1Xhb7qzw7rB1teLH9P2,None,"{'language': 'ko', 'isapp': 'true', 'osversion...",[]
199998,2023-06-21T09:57:12.261Z,launch_app,vBiRfuO1c1Xhb7qzw7rB1teLH9P2,None,"{'language': 'ko', 'isapp': 'true', 'osversion...",[]


In [6]:
def expand_dict_columns(df, dict_columns):
    df = df.copy()
    for col in dict_columns:
        # # 문자열인 dict 컬럼을 실제 dict로 변환
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

        # dict 컬럼을 여러 개의 컬럼으로 확장
        expanded = df[col].apply(pd.Series)

        # 컬럼 이름에 원래 컬럼명을 prefix로 추가
        expanded.columns = [f"{col}_{k}" for k in expanded.columns]

        # 원래 컬럼 제거하고 확장된 컬럼 병합
        df = pd.concat([df.drop(columns=[col]), expanded], axis=1)
    return df

log6_2 = expand_dict_columns(log6_1, ['hackle_properties', 'event_properties'])

In [7]:
log6_2

,Asia/Seoul,event_key,session_id,user_id,hackle_properties_language,hackle_properties_isapp,hackle_properties_osversion,hackle_properties_devicevendor,hackle_properties_versionname,hackle_properties_osname,hackle_properties_platform,hackle_properties_devicemodel,event_properties_0
0,2023-06-18T00:07:08.014Z,$session_start,5cmi6ko54fPkNcJz6OftyN04nxL2,None,ko-KR,true,16.4.1,Apple,1.2.10,iOS,Mobile,"iPhone15,2",NaN
1,2023-06-18T00:07:11.039Z,$session_end,5cmi6ko54fPkNcJz6OftyN04nxL2,None,ko-KR,true,16.5,Apple,1.2.15,iOS,Mobile,"iPhone15,2",NaN
2,2023-06-18T00:07:34.065Z,$session_start,WrOTpJzMF0fUed9MlXQggRmyXbw2,None,ko-KR,true,15.7.3,Apple,1.2.8,iOS,Mobile,"iPhone8,4",NaN
3,2023-06-18T01:01:12.503Z,$session_start,nn7Kppgo3HfNTFkqiDCJsP2aOVI2,None,ko-KR,true,16.3.1,Apple,1.2.10,iOS,Mobile,"iPad13,1",NaN
4,2023-06-18T02:30:47.469Z,$session_start,WrOTpJzMF0fUed9MlXQggRmyXbw2,None,ko-KR,true,15.7.3,Apple,1.2.8,iOS,Mobile,"iPhone8,4",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,2023-06-21T09:57:11.762Z,view_message_tap,f7042063-bd63-4eaa-8347-47095ab58759,JDELTFGyC3annYUuLC1mIumKlJv1,ko,true,9,samsung,1.2.15,Android,Mobile,SM-G950N,NaN
199996,2023-06-21T09:57:11.887Z,view_home_tap,e47237e5-4d80-447a-a36b-a1864583990a,Ex4Ne8WkciTcwuobSOIMJeN9D1C2,ko,true,13,samsung,1.2.15,Android,Mobile,SM-G991N,NaN
199997,2023-06-21T09:57:12.261Z,$session_start,vBiRfuO1c1Xhb7qzw7rB1teLH9P2,None,ko,true,12,samsung,1.2.15,Android,Mobile,SM-G977N,NaN
199998,2023-06-21T09:57:12.261Z,launch_app,vBiRfuO1c1Xhb7qzw7rB1teLH9P2,None,ko,true,12,samsung,1.2.15,Android,Mobile,SM-G977N,NaN


In [9]:
log6_2 = log6_2.drop(columns=['hackle_properties_language','hackle_properties_osversion','hackle_properties_devicevendor',\
    'hackle_properties_osname','hackle_properties_versionname','hackle_properties_platform','hackle_properties_devicemodel',\
        'hackle_properties_isapp'])

In [10]:
log6_2

,Asia/Seoul,event_key,session_id,user_id,event_properties_0
0,2023-06-18T00:07:08.014Z,$session_start,5cmi6ko54fPkNcJz6OftyN04nxL2,None,NaN
1,2023-06-18T00:07:11.039Z,$session_end,5cmi6ko54fPkNcJz6OftyN04nxL2,None,NaN
2,2023-06-18T00:07:34.065Z,$session_start,WrOTpJzMF0fUed9MlXQggRmyXbw2,None,NaN
3,2023-06-18T01:01:12.503Z,$session_start,nn7Kppgo3HfNTFkqiDCJsP2aOVI2,None,NaN
4,2023-06-18T02:30:47.469Z,$session_start,WrOTpJzMF0fUed9MlXQggRmyXbw2,None,NaN
...,...,...,...,...,...
199995,2023-06-21T09:57:11.762Z,view_message_tap,f7042063-bd63-4eaa-8347-47095ab58759,JDELTFGyC3annYUuLC1mIumKlJv1,NaN
199996,2023-06-21T09:57:11.887Z,view_home_tap,e47237e5-4d80-447a-a36b-a1864583990a,Ex4Ne8WkciTcwuobSOIMJeN9D1C2,NaN
199997,2023-06-21T09:57:12.261Z,$session_start,vBiRfuO1c1Xhb7qzw7rB1teLH9P2,None,NaN
199998,2023-06-21T09:57:12.261Z,launch_app,vBiRfuO1c1Xhb7qzw7rB1teLH9P2,None,NaN


In [11]:
log6_2['event_key'].unique()

array(['$session_start', '$session_end', 'launch_app', 'view_home_tap',
       'view_message_tap', 'view_timeline_tap', 'view_profile_tap',
       'view_questions_tap', 'view_friendplus_tap', 'view_shop',
       '$properties', 'click_purchase', 'click_notice',
       'click_question_ask', 'click_question_start',
       'view_question_question', 'click_question_open', 'view_login',
       'click_notice_detail', 'skip_question_question',
       'click_profile_ask', 'complete_question_question',
       'click_question_share', 'view_signup', 'click_friend_invite',
       'click_invite_friend', 'click_autoadd_contact'], dtype=object)

##### 2023-06-23

In [2]:
log7 = pd.read_parquet('gs://sprint_final_project/final_project/hackle_log/2023-06-23.parquet', engine='pyarrow')

In [3]:
log7_1 = log7[log7.index < 200000]

In [4]:
log7_1['user_properties'] = log7_1['user_properties'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

In [5]:
log7_1 = log7_1.drop(columns=['value','id','device_id','user_properties'])
log7_1

,Asia/Seoul,event_key,session_id,user_id,hackle_properties,event_properties
0,2023-06-24T00:00:00.023Z,view_question_question,2b7ad8fc-88ee-409c-9ac0-01e182be56a8,mNLDRM56CjeiREW9NH8dHObLiTM2,"{'language': 'ko', 'isapp': 'true', 'osversion...",[]
1,2023-06-24T00:00:00.064Z,view_timeline_tap,XjKd6fh3CCfKJxobCxGssTIcoCp1,XjKd6fh3CCfKJxobCxGssTIcoCp1,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
2,2023-06-24T00:00:00.148Z,view_message_tap,QepNLzWEkTNvfHSSkhCldJH5eMw2,QepNLzWEkTNvfHSSkhCldJH5eMw2,"{'language': 'ko-US', 'isapp': 'true', 'osvers...",[]
3,2023-06-24T00:00:00.169Z,$session_start,C03B343F-B566-46BA-9030-8E441B71EC33,WVcHhMtN5uVbI9ZW7OM3iMWeXMl1,"{'language': 'ko-US', 'isapp': 'true', 'osvers...",[]
4,2023-06-24T00:00:00.174Z,view_home_tap,C03B343F-B566-46BA-9030-8E441B71EC33,WVcHhMtN5uVbI9ZW7OM3iMWeXMl1,"{'language': 'ko-US', 'isapp': 'true', 'osvers...",[]
...,...,...,...,...,...,...
199995,2023-06-24T02:20:41.801Z,view_questions_tap,zRzQhNicWNbSxAu2vaZyXdqaOJs1,zRzQhNicWNbSxAu2vaZyXdqaOJs1,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
199996,2023-06-24T02:20:41.813Z,view_message_tap,nGZLI1Tyx7eTKw0pns1fFBgUSwE2,nGZLI1Tyx7eTKw0pns1fFBgUSwE2,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
199997,2023-06-24T02:20:41.869Z,view_timeline_tap,PPR40KJh3heo4s6SeWMUF6fTet12,PPR40KJh3heo4s6SeWMUF6fTet12,"{'language': 'en-KR', 'isapp': 'true', 'osvers...",[]
199998,2023-06-24T02:20:41.895Z,view_profile_tap,6cc44d83-e103-48e2-9cac-cf2ac174d9a7,umSamYN0L6YAAnnMPOOqPFTT2lE3,"{'language': 'ko', 'isapp': 'true', 'osversion...",[]


In [6]:
def expand_dict_columns(df, dict_columns):
    df = df.copy()
    for col in dict_columns:
        # # 문자열인 dict 컬럼을 실제 dict로 변환
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

        # dict 컬럼을 여러 개의 컬럼으로 확장
        expanded = df[col].apply(pd.Series)

        # 컬럼 이름에 원래 컬럼명을 prefix로 추가
        expanded.columns = [f"{col}_{k}" for k in expanded.columns]

        # 원래 컬럼 제거하고 확장된 컬럼 병합
        df = pd.concat([df.drop(columns=[col]), expanded], axis=1)
    return df

log7_2 = expand_dict_columns(log7_1, ['hackle_properties', 'event_properties'])

In [7]:
log7_2 = log7_2.drop(columns=['hackle_properties_language','hackle_properties_osversion','hackle_properties_devicevendor',\
    'hackle_properties_osname','hackle_properties_versionname','hackle_properties_platform','hackle_properties_devicemodel',\
        'hackle_properties_isapp'])

In [8]:
log7_2

,Asia/Seoul,event_key,session_id,user_id,event_properties_0
0,2023-06-24T00:00:00.023Z,view_question_question,2b7ad8fc-88ee-409c-9ac0-01e182be56a8,mNLDRM56CjeiREW9NH8dHObLiTM2,NaN
1,2023-06-24T00:00:00.064Z,view_timeline_tap,XjKd6fh3CCfKJxobCxGssTIcoCp1,XjKd6fh3CCfKJxobCxGssTIcoCp1,NaN
2,2023-06-24T00:00:00.148Z,view_message_tap,QepNLzWEkTNvfHSSkhCldJH5eMw2,QepNLzWEkTNvfHSSkhCldJH5eMw2,NaN
3,2023-06-24T00:00:00.169Z,$session_start,C03B343F-B566-46BA-9030-8E441B71EC33,WVcHhMtN5uVbI9ZW7OM3iMWeXMl1,NaN
4,2023-06-24T00:00:00.174Z,view_home_tap,C03B343F-B566-46BA-9030-8E441B71EC33,WVcHhMtN5uVbI9ZW7OM3iMWeXMl1,NaN
...,...,...,...,...,...
199995,2023-06-24T02:20:41.801Z,view_questions_tap,zRzQhNicWNbSxAu2vaZyXdqaOJs1,zRzQhNicWNbSxAu2vaZyXdqaOJs1,NaN
199996,2023-06-24T02:20:41.813Z,view_message_tap,nGZLI1Tyx7eTKw0pns1fFBgUSwE2,nGZLI1Tyx7eTKw0pns1fFBgUSwE2,NaN
199997,2023-06-24T02:20:41.869Z,view_timeline_tap,PPR40KJh3heo4s6SeWMUF6fTet12,PPR40KJh3heo4s6SeWMUF6fTet12,NaN
199998,2023-06-24T02:20:41.895Z,view_profile_tap,6cc44d83-e103-48e2-9cac-cf2ac174d9a7,umSamYN0L6YAAnnMPOOqPFTT2lE3,NaN


##### 2023-06-29

In [ ]:
log8 = pd.read_parquet('gs://sprint_final_project/final_project/hackle_log/2023-06-29.parquet', engine='pyarrow')

In [ ]:
log8_1 = log8[log8.index < 200000]

In [ ]:
log8_1['user_properties'] = log9log8_1_1['user_properties'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

In [ ]:
log8_1 = log8_1.drop(columns=['value','id','device_id','user_properties'])
log8_1

In [ ]:
def expand_dict_columns(df, dict_columns):
    df = df.copy()
    for col in dict_columns:
        # # 문자열인 dict 컬럼을 실제 dict로 변환
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

        # dict 컬럼을 여러 개의 컬럼으로 확장
        expanded = df[col].apply(pd.Series)

        # 컬럼 이름에 원래 컬럼명을 prefix로 추가
        expanded.columns = [f"{col}_{k}" for k in expanded.columns]

        # 원래 컬럼 제거하고 확장된 컬럼 병합
        df = pd.concat([df.drop(columns=[col]), expanded], axis=1)
    return df

log8_2 = expand_dict_columns(log8_1, ['hackle_properties', 'event_properties'])

In [ ]:
log8_2 = log8_2.drop(columns=['hackle_properties_language','hackle_properties_osversion','hackle_properties_devicevendor',\
    'hackle_properties_osname','hackle_properties_versionname','hackle_properties_platform','hackle_properties_devicemodel',\
        'hackle_properties_isapp'])

In [ ]:
log8_2

##### 2023-07-05

In [2]:
log9 = pd.read_parquet('gs://sprint_final_project/final_project/hackle_log/2023-07-05.parquet', engine='pyarrow')

In [3]:
log9_1 = log9[log9.index < 200000]

In [5]:
log9_1['user_properties'] = log9_1['user_properties'].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

In [6]:
log9_1 = log9_1.drop(columns=['value','id','device_id','user_properties'])
log9_1

,Asia/Seoul,event_key,session_id,user_id,hackle_properties,event_properties
0,2023-07-06T00:00:00.234Z,view_home_tap,GCZf2jHnnpSHH7DnviWmLLiG20j1,GCZf2jHnnpSHH7DnviWmLLiG20j1,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
1,2023-07-06T00:00:00.285Z,$session_start,8PZ7GgUaVXSAGn3lPv2E51lfeir1,8PZ7GgUaVXSAGn3lPv2E51lfeir1,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
2,2023-07-06T00:00:00.372Z,view_profile_tap,ZeH6snQ1gxWQfRodLFDV6qMMcmG2,ZeH6snQ1gxWQfRodLFDV6qMMcmG2,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
3,2023-07-06T00:00:00.397Z,launch_app,8PZ7GgUaVXSAGn3lPv2E51lfeir1,8PZ7GgUaVXSAGn3lPv2E51lfeir1,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
4,2023-07-06T00:00:00.473Z,$session_start,raovhvckyKM8m1X2hKrIja8yIb92,raovhvckyKM8m1X2hKrIja8yIb92,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
...,...,...,...,...,...,...
199995,2023-07-06T16:04:56.915Z,view_questions_tap,PAGHJwNS06aIMEXr6tlj3xBhgiE3,PAGHJwNS06aIMEXr6tlj3xBhgiE3,"{'language': 'ko', 'isapp': 'true', 'osversion...",[]
199996,2023-07-06T16:04:56.934Z,view_question_question,CiNafp7sqxWQHTcU3Y3lMCOLQ9C3,CiNafp7sqxWQHTcU3Y3lMCOLQ9C3,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
199997,2023-07-06T16:04:57.183Z,$session_start,1B984B57-8979-47E2-BDB6-A28C5F7D4731,FbUC4XSS8eNqoYaW2KBOqArInEl2,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]
199998,2023-07-06T16:04:57.237Z,view_message_tap,n0zc9eovgIRO1165GpowWahNKIZ2,n0zc9eovgIRO1165GpowWahNKIZ2,"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[]


In [7]:
def expand_dict_columns(df, dict_columns):
    df = df.copy()
    for col in dict_columns:
        # # 문자열인 dict 컬럼을 실제 dict로 변환
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x,str) else x)

        # dict 컬럼을 여러 개의 컬럼으로 확장
        expanded = df[col].apply(pd.Series)

        # 컬럼 이름에 원래 컬럼명을 prefix로 추가
        expanded.columns = [f"{col}_{k}" for k in expanded.columns]

        # 원래 컬럼 제거하고 확장된 컬럼 병합
        df = pd.concat([df.drop(columns=[col]), expanded], axis=1)
    return df

log9_2 = expand_dict_columns(log9_1, ['hackle_properties', 'event_properties'])

In [8]:
log9_2 = log9_2.drop(columns=['hackle_properties_language','hackle_properties_osversion','hackle_properties_devicevendor',\
    'hackle_properties_osname','hackle_properties_versionname','hackle_properties_platform','hackle_properties_devicemodel',\
        'hackle_properties_isapp'])

In [10]:
log9_2

,Asia/Seoul,event_key,session_id,user_id,event_properties_0
0,2023-07-06T00:00:00.234Z,view_home_tap,GCZf2jHnnpSHH7DnviWmLLiG20j1,GCZf2jHnnpSHH7DnviWmLLiG20j1,NaN
1,2023-07-06T00:00:00.285Z,$session_start,8PZ7GgUaVXSAGn3lPv2E51lfeir1,8PZ7GgUaVXSAGn3lPv2E51lfeir1,NaN
2,2023-07-06T00:00:00.372Z,view_profile_tap,ZeH6snQ1gxWQfRodLFDV6qMMcmG2,ZeH6snQ1gxWQfRodLFDV6qMMcmG2,NaN
3,2023-07-06T00:00:00.397Z,launch_app,8PZ7GgUaVXSAGn3lPv2E51lfeir1,8PZ7GgUaVXSAGn3lPv2E51lfeir1,NaN
4,2023-07-06T00:00:00.473Z,$session_start,raovhvckyKM8m1X2hKrIja8yIb92,raovhvckyKM8m1X2hKrIja8yIb92,NaN
...,...,...,...,...,...
199995,2023-07-06T16:04:56.915Z,view_questions_tap,PAGHJwNS06aIMEXr6tlj3xBhgiE3,PAGHJwNS06aIMEXr6tlj3xBhgiE3,NaN
199996,2023-07-06T16:04:56.934Z,view_question_question,CiNafp7sqxWQHTcU3Y3lMCOLQ9C3,CiNafp7sqxWQHTcU3Y3lMCOLQ9C3,NaN
199997,2023-07-06T16:04:57.183Z,$session_start,1B984B57-8979-47E2-BDB6-A28C5F7D4731,FbUC4XSS8eNqoYaW2KBOqArInEl2,NaN
199998,2023-07-06T16:04:57.237Z,view_message_tap,n0zc9eovgIRO1165GpowWahNKIZ2,n0zc9eovgIRO1165GpowWahNKIZ2,NaN


In [14]:
log9_2['event_properties_0'].unique()

array([nan, ('page_name', 'home'), ('page_name', '학교선택'),
       ('page_name', '학년선택'), ('page_name', 'invite'),
       ('page_name', 'notice'), ('$clearall', '{"clearall":"-"}'),
       ('$set', '{"heart_balance":"5932"}'), ('page_name', '반선택'),
       ('page_name', '연락처'), ('page_name', '번호입력'),
       ('$set', '{"heart_balance":"487"}'), ('page_name', 'profile'),
       ('$set', '{"heart_balance":"44"}'), ('page_name', '번호인증'),
       ('page_name', '이름입력'), ('page_name', '아이디입력'),
       ('page_name', '성별선택'), ('page_name', '프사설정'),
       ('page_name', '알수도있는친구'), ('$set', '{"heart_balance":"116"}'),
       ('item_name', '777 하트'), ('page_name', '푸쉬'), ('page_name', '웰컴'),
       ('$set', '{"heart_balance":"5178"}'),
       ('$set', '{"heart_balance":"41"}'), ('item_name', '무료충전소'),
       ('$set', '{"heart_balance":"9335"}'),
       ('$set', '{"heart_balance":"132"}'),
       ('$set', '{"heart_balance":"5440"}'),
       ('$set', '{"heart_balance":"82"}'), ('item_name', '200 하트'),


In [15]:
log9_2['event_key'].unique()

array(['view_home_tap', '$session_start', 'view_profile_tap',
       'launch_app', '$session_end', 'skip_question_question',
       'view_questions_tap', 'view_timeline_tap', 'click_question_start',
       'view_question_question', 'view_message_tap',
       'click_question_share', 'click_question_open', 'view_signup',
       'click_friend_invite', 'view_friendplus_tap', 'click_notice',
       'complete_question_question', 'click_invite_friend',
       'click_notice_detail', '$properties', 'view_shop',
       'click_question_ask', 'view_login', 'click_profile_ask',
       'complete_signup', 'click_purchase', 'click_autoadd_contact'],
      dtype=object)